In [1]:
# Это нужно, чтобы можно было работать с файлами в Django
import sys 
import os, django

# Домашний комп
if os.path.isdir("C:\\IT\\python\\django_sbx\\django_sbx\\django_sbx"):
    sys.path.append("C:\\IT\\python\\django_sbx\\django_sbx\\django_sbx")
# Рабочий комп 
elif os.path.isdir("C:\\Users\\rustem.diveev\\Desktop\\django_sbx\\django_sbx"):
    sys.path.append("C:\\Users\\rustem.diveev\\Desktop\\django_sbx\\django_sbx")
    
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "django_sbx.settings")
django.setup()

# Это нужно, чтобы разрешить синхронные операции в асинхронном окружении - ничего не понял
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [2]:
from django.db import transaction 

from transaction_management.models import (
    TransactionTutorialModelDefault, TransactionTutorialModelAdditionalDB
)

In [3]:
"""
    Статическая модель зарегистрированная в default 
    Вставляем 1 строку
"""
TransactionTutorialModelDefault.objects.all().delete()

row = TransactionTutorialModelDefault(
    attr_u="A",
    attr_nn="A",
    attr_ut_1="A",
    attr_ut_2="A"
)
row.save()

In [4]:
"""
    Пытаемся вставить 2 строки, 1 - другая, 2 - с ранее повторяющимися значениями
    Проверяем, что транзакция откатится
"""
try:
    with transaction.atomic():
        row = TransactionTutorialModelDefault(
            attr_u="B",
            attr_nn="B",
            attr_ut_1="B",
            attr_ut_2="B"
        )
        row.save()
        row = TransactionTutorialModelDefault(
            attr_u="A",
            attr_nn="A",
            attr_ut_1="A",
            attr_ut_2="A"
        )
        row.save()
except Exception:
    pass


In [5]:
"""
    Действительно, видна только 1 строка
"""
TransactionTutorialModelDefault.objects.all().count()

1

In [6]:
"""
    Теперь следующий прототип: пытаемся сохранить несколько строк, где все, кроме одной - невалидные 
    Необходимо записать все ошибки в массив и откатить транзакцию
"""
from django.db import DatabaseError

row_valid = TransactionTutorialModelDefault(
    attr_u="B",
    attr_nn="B",
    attr_ut_1="B",
    attr_ut_2="B"
)

row_invalid_u = TransactionTutorialModelDefault(
    attr_u="A",
    attr_nn="A1",
    attr_ut_1="A1",
    attr_ut_2="A1"
)

row_invalid_nn = TransactionTutorialModelDefault(
    attr_u="C",
    attr_nn=None,
    attr_ut_1="A2",
    attr_ut_2="A2"
)

row_invalid_ut = TransactionTutorialModelDefault(
    attr_u="D",
    attr_nn="D",
    attr_ut_1="A",
    attr_ut_2="A"
)

rows = [row_valid, row_invalid_u, row_invalid_nn, row_invalid_ut]
errors = []

"""
    Так нельзя, к сожалению - последние ошибки не выводятся
"""
with transaction.atomic():
    for row in rows:
        try:
            row.save()
        except DatabaseError as err:
            errors.append(err)

In [7]:
errors

[django.db.utils.IntegrityError('ОШИБКА:  повторяющееся значение ключа нарушает ограничение уникальности "transaction_management_transactiontutorialmodeldefau_attr_u_key"\nDETAIL:  Ключ "(attr_u)=(A)" уже существует.\n'),
 django.db.transaction.TransactionManagementError("An error occurred in the current transaction. You can't execute queries until the end of the 'atomic' block."),
 django.db.transaction.TransactionManagementError("An error occurred in the current transaction. You can't execute queries until the end of the 'atomic' block.")]

In [8]:
errors = []

"""
    Так работает и собирает все ошибки, но при этом не откатывает если что-то пошло не так
"""
for row in rows:
    try:
        row.save()
    except DatabaseError as err:
        errors.append(err)
        
errors

[django.db.utils.IntegrityError('ОШИБКА:  повторяющееся значение ключа нарушает ограничение уникальности "transaction_management_transactiontutorialmodeldefau_attr_u_key"\nDETAIL:  Ключ "(attr_u)=(A)" уже существует.\n'),
 django.db.utils.IntegrityError('ОШИБКА:  значение NULL в столбце "attr_nn" отношения "transaction_management_transactiontutorialmodeldefault" нарушает ограничение NOT NULL\nDETAIL:  Ошибочная строка содержит (63, C, null, A2, A2).\n'),
 django.db.utils.IntegrityError('ОШИБКА:  повторяющееся значение ключа нарушает ограничение уникальности "transaction_management_t_attr_ut_1_attr_ut_2_1e765a22_uniq"\nDETAIL:  Ключ "(attr_ut_1, attr_ut_2)=(A, A)" уже существует.\n')]

In [9]:
TransactionTutorialModelDefault.objects.exclude(attr_u="A").delete()

(1, {'transaction_management.TransactionTutorialModelDefault': 1})

In [10]:
"""
    Пробуем собрать все ошибки и откатить транзакцию целиком, если есть хотя бы 1 ошибка
    Вот это работает, и это нереально здорово
"""
errors = []

try:
    with transaction.atomic():
        for row in rows:
            with transaction.atomic():
                try:
                    row.save()
                except DatabaseError as err:
                    errors.append(err)
        if errors:
            raise DatabaseError
except DatabaseError:
    pass

In [11]:
errors

[django.db.utils.IntegrityError('ОШИБКА:  повторяющееся значение ключа нарушает ограничение уникальности "transaction_management_transactiontutorialmodeldefau_attr_u_key"\nDETAIL:  Ключ "(attr_u)=(A)" уже существует.\n'),
 django.db.utils.IntegrityError('ОШИБКА:  значение NULL в столбце "attr_nn" отношения "transaction_management_transactiontutorialmodeldefault" нарушает ограничение NOT NULL\nDETAIL:  Ошибочная строка содержит (66, C, null, A2, A2).\n'),
 django.db.utils.IntegrityError('ОШИБКА:  повторяющееся значение ключа нарушает ограничение уникальности "transaction_management_t_attr_ut_1_attr_ut_2_1e765a22_uniq"\nDETAIL:  Ключ "(attr_ut_1, attr_ut_2)=(A, A)" уже существует.\n')]

In [12]:
"""
    Пробуем выполнить то же самое с моделью в другой БД (не default)
"""
TransactionTutorialModelAdditionalDB.objects.db_manager("additional_db").all().delete()

row = TransactionTutorialModelAdditionalDB(
    attr_u="A",
    attr_nn="A",
    attr_ut_1="A",
    attr_ut_2="A"
).save(using="additional_db")


In [13]:
row_valid = TransactionTutorialModelAdditionalDB(
    attr_u="B",
    attr_nn="B",
    attr_ut_1="B",
    attr_ut_2="B"
)

row_invalid_u = TransactionTutorialModelAdditionalDB(
    attr_u="A",
    attr_nn="A1",
    attr_ut_1="A1",
    attr_ut_2="A1"
)

row_invalid_nn = TransactionTutorialModelAdditionalDB(
    attr_u="C",
    attr_nn=None,
    attr_ut_1="A2",
    attr_ut_2="A2"
)

row_invalid_ut = TransactionTutorialModelAdditionalDB(
    attr_u="D",
    attr_nn="D",
    attr_ut_1="A",
    attr_ut_2="A"
)

rows = [row_valid, row_invalid_u, row_invalid_nn, row_invalid_ut]
errors = []

try:
    with transaction.atomic():
        for row in rows:
            with transaction.atomic():
                try:
                    row.save(using="additional_db")
                except DatabaseError as err:
                    errors.append(err)
        if errors:
            raise DatabaseError
except DatabaseError:
    pass

errors


[django.db.utils.IntegrityError('ОШИБКА:  повторяющееся значение ключа нарушает ограничение уникальности "transaction_management_transactiontutorialmodeladdit_attr_u_key"\nDETAIL:  Ключ "(attr_u)=(A)" уже существует.\n'),
 django.db.utils.IntegrityError('ОШИБКА:  значение NULL в столбце "attr_nn" отношения "transaction_management_transactiontutorialmodeladditionaldb" нарушает ограничение NOT NULL\nDETAIL:  Ошибочная строка содержит (9, C, null, A2, A2).\n'),
 django.db.utils.IntegrityError('ОШИБКА:  повторяющееся значение ключа нарушает ограничение уникальности "transaction_management_t_attr_ut_1_attr_ut_2_4ce65a96_uniq"\nDETAIL:  Ключ "(attr_ut_1, attr_ut_2)=(A, A)" уже существует.\n')]